Load and Merge Datasets
(Have merged the dataset using SQLLite because as the dataset is too large kernal crashes. But SQLLite is good in handling these type of data it is used)

In [8]:
import pandas as pd
import sqlite3
from datetime import datetime

# Read the CSV files and convert timestamps
ec2_df = pd.read_csv('ec2_data.csv')
rds_df = pd.read_csv('rds_data.csv')
ecs_df = pd.read_csv('ecs_data.csv')

# Convert timestamps
ec2_df['timestamp'] = pd.to_datetime(ec2_df['timestamp'])
rds_df['timestamp'] = pd.to_datetime(rds_df['timestamp'])
ecs_df['timestamp'] = pd.to_datetime(ecs_df['timestamp'])

# Create database connection
conn = sqlite3.connect('merged_metrics.db')

# Store dataframes in SQLite
ec2_df.to_sql('ec2_data', conn, if_exists='replace', index=False)
rds_df.to_sql('rds_data', conn, if_exists='replace', index=False)
ecs_df.to_sql('ecs_data', conn, if_exists='replace', index=False)

# Query with correct column names
query = """
WITH all_timestamps AS (
    SELECT DISTINCT timestamp 
    FROM (
        SELECT timestamp FROM ec2_data
        UNION ALL
        SELECT timestamp FROM rds_data
        UNION ALL
        SELECT timestamp FROM ecs_data
    )
)
SELECT 
    at.timestamp,
    ec2.EC2_CPUUtilization,
    ec2.EC2_MemoryUtilization,
    ec2.EC2_DiskWriteOps,
    ec2.EC2_NetworkIn,
    rds.RDS_CPUUtilization,
    rds.RDS_FreeableMemory,
    rds.RDS_DatabaseConnections,
    rds.RDS_WriteIOPS,
    ecs.ECS_CPUUtilization,
    ecs.ECS_MemoryUtilization,
    ecs.ECS_RunningTaskCount
FROM all_timestamps at
LEFT JOIN ec2_data ec2 ON at.timestamp = ec2.timestamp
LEFT JOIN rds_data rds ON at.timestamp = rds.timestamp
LEFT JOIN ecs_data ecs ON at.timestamp = ecs.timestamp
ORDER BY at.timestamp;
"""

# Execute query and load results
df = pd.read_sql_query(query, conn)

# Remove any duplicate rows
df = df.drop_duplicates()

# Close database connection
conn.close()

# Save the merged dataset
df.to_csv('merged_cloud_metrics.csv', index=False)

print("\nMerged dataset saved as 'merged_cloud_metrics.csv'")
print("\nColumns in the dataset:")
print(df.columns.tolist())
print("\nSample of merged data:")
print(df.head())


Merged dataset saved as 'merged_cloud_metrics.csv'

Columns in the dataset:
['timestamp', 'EC2_CPUUtilization', 'EC2_MemoryUtilization', 'EC2_DiskWriteOps', 'EC2_NetworkIn', 'RDS_CPUUtilization', 'RDS_FreeableMemory', 'RDS_DatabaseConnections', 'RDS_WriteIOPS', 'ECS_CPUUtilization', 'ECS_MemoryUtilization', 'ECS_RunningTaskCount']

Sample of merged data:
             timestamp  EC2_CPUUtilization  EC2_MemoryUtilization  \
0                 None                 NaN                    NaN   
1  2023-01-01 00:00:09           63.282078                    NaN   
2  2023-01-01 00:00:15                 NaN                    NaN   
3  2023-01-01 00:00:31                 NaN                    NaN   
4  2023-01-01 00:00:36           93.148608              68.979072   

   EC2_DiskWriteOps  EC2_NetworkIn  RDS_CPUUtilization  RDS_FreeableMemory  \
0               NaN            NaN                 NaN                 NaN   
1            3983.0     212.708467                 NaN                 

Load the Mapped Dataset

In [9]:
# Load the mapped dataset
df = pd.read_csv('merged_cloud_metrics.csv')

# Display the first few rows
print(df.head())

             timestamp  EC2_CPUUtilization  EC2_MemoryUtilization  \
0                  NaN                 NaN                    NaN   
1  2023-01-01 00:00:09           63.282078                    NaN   
2  2023-01-01 00:00:15                 NaN                    NaN   
3  2023-01-01 00:00:31                 NaN                    NaN   
4  2023-01-01 00:00:36           93.148608              68.979072   

   EC2_DiskWriteOps  EC2_NetworkIn  RDS_CPUUtilization  RDS_FreeableMemory  \
0               NaN            NaN                 NaN                 NaN   
1            3983.0     212.708467                 NaN                 NaN   
2               NaN            NaN           30.719806           10.800739   
3               NaN            NaN           14.019569           25.632749   
4            2085.0     621.709971                 NaN                 NaN   

   RDS_DatabaseConnections  RDS_WriteIOPS  ECS_CPUUtilization  \
0                      NaN            NaN          

Handle missing values

In [10]:
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Step 4: Handle missing values
print(df.isnull().sum())
df.fillna(df.mean(), inplace=True)

# Step 5: Save the cleaned dataset
df.to_csv('cleaned_cloud_metrics.csv', index=False)

timestamp                        1
EC2_CPUUtilization         1199400
EC2_MemoryUtilization      1199708
EC2_DiskWriteOps           1199367
EC2_NetworkIn              1199426
RDS_CPUUtilization         1198469
RDS_FreeableMemory         1198979
RDS_DatabaseConnections    1138788
RDS_WriteIOPS              1199129
ECS_CPUUtilization         1198765
ECS_MemoryUtilization      1199061
ECS_RunningTaskCount       1138769
dtype: int64


Feature Engineering
1. Temporal features
2. Rolling Averages
3. Lagged features
4. Utilization ratios

In [11]:
import numpy as np

# Load the preprocessed dataset
df = pd.read_csv('cleaned_cloud_metrics.csv')

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 1. Temporal Features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['timestamp'].dt.month

# 2. Rolling Averages (e.g., 5-time-step rolling average for CPU and memory)
window_size = 5
df['cpu_rolling_avg'] = df['EC2_CPUUtilization'].rolling(window=window_size).mean()
df['memory_rolling_avg'] = df['EC2_MemoryUtilization'].rolling(window=window_size).mean()

# Fill NaN values in rolling averages (first few rows)
df.fillna(method='bfill', inplace=True)  # Backward fill

# 3. Lagged Features (e.g., CPU usage at t-1, t-2)
df['cpu_lag_1'] = df['EC2_CPUUtilization'].shift(1)
df['cpu_lag_2'] = df['EC2_CPUUtilization'].shift(2)

# Fill NaN values in lagged features
df.bfill(inplace=True)  # Backward fill

# 4. Utilization Ratios
df['cpu_memory_ratio'] = df['EC2_CPUUtilization'] / df['EC2_MemoryUtilization']
df['network_in_out_ratio'] = df['EC2_NetworkIn'] / df['EC2_NetworkOut']

# Handle division by zero (replace infinite values with NaN and then fill with 0)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Display the new features
print(df.head())

ValueError: time data "2023-01-01 23:25:12" doesn't match format "%Y-%m-%d %H:%M:%S.%f", at position 8333. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

# Sort by timestamp (ensure data is in chronological order)
df.sort_values('timestamp', inplace=True)

# Define features (X) and target (y)
# For LSTM: Target could be future CPU usage (e.g., next time step)
X = df.drop(columns=['timestamp'])  # Drop timestamp (not a feature)
y = df['EC2_CPUUtilization']  # Example target (can be adjusted)

# Time-based split (e.g., 70% train, 15% validation, 15% test)
train_size = int(0.7 * len(df))
val_size = int(0.15 * len(df))

X_train, X_val_test = X[:train_size], X[train_size:]
y_train, y_val_test = y[:train_size], y[train_size:]

X_val, X_test = X_val_test[:val_size], X_val_test[val_size:]
y_val, y_test = y_val_test[:val_size], y_val_test[val_size:]

# Verify the splits
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Testing set size: {len(X_test)}")

Normalize the Data

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Normalize the target (if needed)
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = scaler.transform(y_val.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

Reshape Data for LSTM

In [ ]:
import numpy as np

# Function to create sequences
def create_sequences(data, targets, time_steps=10):
    X_seq, y_seq = [], []
    for i in range(len(data) - time_steps):
        X_seq.append(data[i:i+time_steps])
        y_seq.append(targets[i+time_steps])  # Predict the next CPU usage
    return np.array(X_seq), np.array(y_seq)

# Define time steps (e.g., 10 time steps per sequence)
time_steps = 10

# Create sequences for training, validation, and testing
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)

# Verify the shapes
print(f"Training sequences: {X_train_seq.shape}, Targets: {y_train_seq.shape}")
print(f"Validation sequences: {X_val_seq.shape}, Targets: {y_val_seq.shape}")
print(f"Testing sequences: {X_test_seq.shape}, Targets: {y_test_seq.shape}")

Train LSTM model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential()

# Add LSTM layers
model.add(LSTM(100, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.3))  # Increased dropout rate
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.3))

# Add a Dense output layer
model.add(Dense(1))  # Output layer (predicts a single value)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=20,  # Number of epochs
    batch_size=32,  # Batch size
    verbose=1
)

Evaluate

In [ ]:
# Evaluate on the validation set
val_loss = model.evaluate(X_val_seq, y_val_seq, verbose=0)
print(f"Validation Loss: {val_loss}")

# Evaluate on the testing set
test_loss = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f"Testing Loss: {test_loss}")

# Make predictions
y_pred = model.predict(X_test_seq)

# Inverse transform the predictions and targets to original scale
y_pred_original = scaler.inverse_transform(y_pred)
y_test_original = scaler.inverse_transform(y_test_seq)

# Display some predictions
for i in range(5):
    print(f"Predicted: {y_pred_original[i][0]}, Actual: {y_test_original[i][0]}")

Save model